In [1]:
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    c.execute('''CREATE TABLE sensors(date TEXT, 
                city TEXT, code TEXT, sensor_id REAL, temperature REAL);''')
    c.execute("INSERT INTO sensors VALUES ('2018-02-06', 'Utrecht', 'Red', 42, 15.14)")
    c.execute("SELECT * FROM sensors")
    print(c.fetchone())
    con.execute("DROP TABLE sensors")
    print(c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type='table'").fetchone()[0])
    c.close()

('2018-02-06', 'Utrecht', 'Red', 42.0, 15.14)
0


In [3]:
import sqlite3

con = sqlite3.connect(":memory:")
c = con.cursor()
c.execute('''CREATE TABLE COMPANY
       (ID INT PRIMARY KEY     NOT NULL,
       NAME           TEXT    NOT NULL,
       AGE            INT     NOT NULL,
       ADDRESS        CHAR(50),
       SALARY         REAL)''')
c.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (1, 'Paul', 32, 'California', 20000.00 )")
c.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) \
      VALUES (2, 'Allen', 25, 'Texas', 15000.00 )")
cursor = c.execute("SELECT id, name, address, salary  from COMPANY")
for row in cursor:
   print("ID = ", row[0])
   print("NAME = ", row[1])
   print("ADDRESS = ", row[2])
   print("SALARY = ", row[3], "\n")

ID =  1
NAME =  Paul
ADDRESS =  California
SALARY =  20000.0 

ID =  2
NAME =  Allen
ADDRESS =  Texas
SALARY =  15000.0 



# Accessing database from Pandas

In [5]:
import statsmodels.api as sm
from pandas.io.sql import read_sql
import sqlite3

In [7]:
with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    data_loader = sm.datasets.sunspots.load_pandas()
    df = data_loader.data
    rows = [tuple(x) for x in df.values]
    print(rows)
    
    con.execute("CREATE TABLE sunspots(year, sunactivity)")
    con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?,?)", rows)
    c.execute("SELECT COUNT(*) FROM sunspots")
    print(c.fetchone())
    print("Deleted", con.execute("DELETE FROM sunspots where sunactivity > 20").rowcount, "rows")
    
    c.close()

[(1700.0, 5.0), (1701.0, 11.0), (1702.0, 16.0), (1703.0, 23.0), (1704.0, 36.0), (1705.0, 58.0), (1706.0, 29.0), (1707.0, 20.0), (1708.0, 10.0), (1709.0, 8.0), (1710.0, 3.0), (1711.0, 0.0), (1712.0, 0.0), (1713.0, 2.0), (1714.0, 11.0), (1715.0, 27.0), (1716.0, 47.0), (1717.0, 63.0), (1718.0, 60.0), (1719.0, 39.0), (1720.0, 28.0), (1721.0, 26.0), (1722.0, 22.0), (1723.0, 11.0), (1724.0, 21.0), (1725.0, 40.0), (1726.0, 78.0), (1727.0, 122.0), (1728.0, 103.0), (1729.0, 73.0), (1730.0, 47.0), (1731.0, 35.0), (1732.0, 11.0), (1733.0, 5.0), (1734.0, 16.0), (1735.0, 34.0), (1736.0, 70.0), (1737.0, 81.0), (1738.0, 111.0), (1739.0, 101.0), (1740.0, 73.0), (1741.0, 40.0), (1742.0, 20.0), (1743.0, 16.0), (1744.0, 5.0), (1745.0, 11.0), (1746.0, 22.0), (1747.0, 40.0), (1748.0, 60.0), (1749.0, 80.9), (1750.0, 83.4), (1751.0, 47.7), (1752.0, 47.8), (1753.0, 30.7), (1754.0, 12.2), (1755.0, 9.6), (1756.0, 10.2), (1757.0, 32.4), (1758.0, 47.6), (1759.0, 54.0), (1760.0, 62.9), (1761.0, 85.9), (1762.0, 61.

# SQLAlchemy

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine, UniqueConstraint

In [2]:
Base = declarative_base()

class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    name = Column(String(14), nullable=False, unique=True)
    
    def __repr__(self):
        return "Id=%d name=%s" % (self.id, self.name)
    
class Sensor(Base):
    __tablename__ = 'sensor'
    id = Column(Integer, primary_key=True)
    observation = Column(Integer)
    multiplier = Column(Float)
    station_id = Column(Integer, ForeignKey('station.id'))
    station = relationship(Station)
    
    def __repr__(self):
        return "Id=%d oberservation=%d multiplier=%.1f station_id=%d" % \
                (self.id, self.observation, self.multiplier, self.station_id)

Sensor.__table__

Table('sensor', MetaData(bind=None), Column('id', Integer(), table=<sensor>, primary_key=True, nullable=False), Column('observation', Integer(), table=<sensor>), Column('multiplier', Float(), table=<sensor>), Column('station_id', Integer(), ForeignKey('station.id'), table=<sensor>), schema=None)

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
from pandas.io.sql import read_sql

def populate(session):
    de_bilt = Station(name='De Bilt')
    session.add(de_bilt)
    session.add(Station(name='Utrecht'))
    session.commit()
    print("Station", de_bilt)
    
    temp_sensor = Sensor(observation=20, multiplier=.1, station=de_bilt)
    session.add(temp_sensor)
    session.commit()
    print("Sensor", temp_sensor)

In [4]:
Station.__table__

Table('station', MetaData(bind=None), Column('id', Integer(), table=<station>, primary_key=True, nullable=False), Column('name', String(length=14), table=<station>, nullable=False), schema=None)

In [6]:
engine = create_engine("sqlite:///test.db", echo=True)
Base.metadata.create_all(engine)
#Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()
station = Station(name='test')
session.add(station)
test_query = session.query(Station).filter_by(name='test').first() 
print(test_query)

2018-04-21 00:01:21,702 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-21 00:01:21,706 INFO sqlalchemy.engine.base.Engine ()
2018-04-21 00:01:21,708 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-21 00:01:21,711 INFO sqlalchemy.engine.base.Engine ()
2018-04-21 00:01:21,714 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("station")
2018-04-21 00:01:21,716 INFO sqlalchemy.engine.base.Engine ()
2018-04-21 00:01:21,720 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("sensor")
2018-04-21 00:01:21,723 INFO sqlalchemy.engine.base.Engine ()
2018-04-21 00:01:21,727 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE station (
	id INTEGER NOT NULL, 
	name VARCHAR(14) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2018-04-21 00:01:21,731 INFO sqlalchemy.engine.base.Engine ()
2018-04-21 00:01:21,737 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-21 00:01:21,742 INFO sqlalche

In [7]:
populate(session)

2018-04-21 00:01:36,693 INFO sqlalchemy.engine.base.Engine INSERT INTO station (name) VALUES (?)
2018-04-21 00:01:36,696 INFO sqlalchemy.engine.base.Engine ('De Bilt',)
2018-04-21 00:01:36,699 INFO sqlalchemy.engine.base.Engine INSERT INTO station (name) VALUES (?)
2018-04-21 00:01:36,702 INFO sqlalchemy.engine.base.Engine ('Utrecht',)
2018-04-21 00:01:36,707 INFO sqlalchemy.engine.base.Engine COMMIT
Station 2018-04-21 00:01:36,713 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-21 00:01:36,715 INFO sqlalchemy.engine.base.Engine SELECT station.id AS station_id, station.name AS station_name 
FROM station 
WHERE station.id = ?
2018-04-21 00:01:36,717 INFO sqlalchemy.engine.base.Engine (2,)
Id=2 name=De Bilt
2018-04-21 00:01:36,725 INFO sqlalchemy.engine.base.Engine INSERT INTO sensor (observation, multiplier, station_id) VALUES (?, ?, ?)
2018-04-21 00:01:36,728 INFO sqlalchemy.engine.base.Engine (20, 0.1, 2)
2018-04-21 00:01:36,732 INFO sqlalchemy.engine.base.Engine COMMIT
Se

In [8]:
test_query is station

True

In [9]:
station_query = session.query(Station).filter_by(name='De Bilt').first()
print(session.query(Station).all())
print(session.query(Sensor).all())
print(session.query(Sensor).filter(Sensor.station==station_query).one())
print(read_sql("SELECT * FROM station", engine.raw_connection()))

try:
    os.remove('test.db')
    print('DB deleted')
except OSError:
    print('DB delete failed')

2018-04-21 00:08:01,283 INFO sqlalchemy.engine.base.Engine SELECT station.id AS station_id, station.name AS station_name 
FROM station 
WHERE station.name = ?
 LIMIT ? OFFSET ?
2018-04-21 00:08:01,286 INFO sqlalchemy.engine.base.Engine ('De Bilt', 1, 0)
2018-04-21 00:08:01,291 INFO sqlalchemy.engine.base.Engine SELECT station.id AS station_id, station.name AS station_name 
FROM station
2018-04-21 00:08:01,296 INFO sqlalchemy.engine.base.Engine ()
[Id=1 name=test, Id=2 name=De Bilt, Id=3 name=Utrecht]
2018-04-21 00:08:01,301 INFO sqlalchemy.engine.base.Engine SELECT sensor.id AS sensor_id, sensor.observation AS sensor_observation, sensor.multiplier AS sensor_multiplier, sensor.station_id AS sensor_station_id 
FROM sensor
2018-04-21 00:08:01,303 INFO sqlalchemy.engine.base.Engine ()
[Id=1 oberservation=20 multiplier=0.1 station_id=2]
2018-04-21 00:08:01,309 INFO sqlalchemy.engine.base.Engine SELECT sensor.id AS sensor_id, sensor.observation AS sensor_observation, sensor.multiplier AS sen